# Exam - Advanced Methods in Applied Statistics 2024 - Emilie Jessen

## Problem 5 

In [7]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import scipy 
from iminuit import Minuit
import nestle

In [3]:
# Set som plotting standards:
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 18}
mpl.rc('font', **font)

axes = {'facecolor': 'ghostwhite'}
mpl.rc('axes', **axes)

# Add grid
mpl.rc('axes', grid=True)

# Set custom color cycle
custom_colors= ['dodgerblue', 'red', 'limegreen', 'orange', 'orchid', 'black', 'slategrey', 
                'navy', 'magenta', 'forestgreen', 'lightblue', 'maroon', 'gold', 'lightcoral', 
                'mediumseagreen', 'darkorange', 'darkviolet', 'dimgray', 'darkblue', 'darkred',]
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=custom_colors)

# Set inside tickmarks
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['ytick.minor.visible'] = True

In [4]:
save_plots = True
np.random.seed(42)

In [1]:
def variance(data):
    """Function to calculate the mean and variance of a data set.  
    The bias corrected variance is calculated by dividing by N-1 instead of N. """

    N = len(data)
    mean = np.mean(data)
    var_biased = np.sum((data - mean)**2) / N
    var_unbiased = np.sum((data - mean)**2) / (N - 1)
    
    return mean, var_biased, var_unbiased

In [ ]:
def likelihood(data, func, params):
    """Function to calculate the likelihood of a model given the data. 
    The likelihood is calculated as the product of the probabilities of the data points given the model. """

    likelihood = np.prod(func(data, *params))

    return likelihood

In [2]:
def raster_scan(data, range1, range2, likelihood, func, step=100): 
    """Function to perform a raster scan over the parameter space inside the ranges.
    The likelihood is calculated for each point in the grid and the maximum likelihood is found.
    The function returns the likelihood grid, maximum likelihood and the parameters that give the maximum likelihood."""

    # Create a grid of parameter values
    x = np.linspace(range1[0], range1[1], step)
    y = np.linspace(range2[0], range2[1], step)
    X, Y = np.meshgrid(x, y)
    Z = np.zeros_like(X)

    # Evaluate the likelihood at each point in the grid
    for i in range(len(x)):
        for j in range(len(y)):
            Z[j, i] = likelihood(data, func, [x[i], y[j]])

    # Find the maximum likelihood
    max_likelihood = np.max(Z)
    idx = np.unravel_index(np.argmax(Z, axis=None), Z.shape)
    max_params = [x[idx[0]], y[idx[1]]]

    return Z, max_likelihood, max_params

In [3]:
def plot_raster(Z, range1, range2, max_params, fig, ax):
    """Function to plot the raster scan results. """

    c = ax.pcolormesh(range1, range2, Z, cmap='viridis')
    ax.set_title('Raster scan')
    ax.scatter(max_params[0], max_params[1], marker='x', color='red', label='Max likelihood')
    ax.legend()
    fig.colorbar(c, label='Likelihood')